In [29]:
#!pwd
import pandas as pd
import os
import string
from nltk.corpus import stopwords
from nltk import word_tokenize, WordNetLemmatizer
from nltk import stem, pos_tag
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import os
import re
cwd = os.getcwd()
filepath = cwd+'/data/df_test_indian.csv'
outputfilepath = cwd+'/data/df_test_indian_with_negation.csv'

In [30]:
df = pd.read_csv(filepath)
df.head()
# ffile1 = open(filepath,"r", encoding = "ISO-8859-1")
# df_standard = pd.read_csv(ffile1, encoding = "utf-8")
# df_standard.drop(['restaurant_id', 'date', 'review_id'], inplace=True, axis=1)
# df_standard.rename(columns={'text': 'review', 'Sentiment': 'sentiment'}, inplace=True)
# df_standard.head()

# mask = df_standard.sentiment == 'negative'
# column_name = 'sentiment'
# df_standard.loc[mask, column_name] = -1
# mask = df_standard.sentiment == 'positive'
# column_name = 'sentiment'
# df_standard.loc[mask, column_name] = 1
# df_standard = df_standard.rename(columns = {'stars':'rating'})
# df_standard.head()

# df = pd.concat([df, df_standard], axis=0)
# df = df.reset_index().drop('index', axis=1)

,category,id,name,rating,review,sentiment
0,indian,NKGOw3iT06Wsj2uLOsYTgQ,Dishoom,5,I loved this place. It's conveniently located ...,1
1,indian,NKGOw3iT06Wsj2uLOsYTgQ,Dishoom,4,Really wonderful evening spent at Dishoom in k...,1
2,indian,NKGOw3iT06Wsj2uLOsYTgQ,Dishoom,4,"Food is legendary as ever, great value for mon...",1
3,indian,NKGOw3iT06Wsj2uLOsYTgQ,Dishoom,5,I've been in London for only a week and I've a...,1
4,indian,NKGOw3iT06Wsj2uLOsYTgQ,Dishoom,5,"When in London, do what the desis do, right?! ...",1


In [31]:
ratings = df['rating']
sentiments = df['sentiment']
reviews = df['review']

In [32]:
sentiment_trans = []
for r in ratings.keys():
    rating = ratings.loc[r]
    sentiment = sentiments.loc[r]
    sentiment = int(sentiment)
    if rating <= 3:
        sentiment = -1
    else:
        sentiment = sentiment
    sentiment_trans.insert(r, sentiment)
sentiments = pd.Series(sentiment_trans, index=ratings.keys())

df = pd.concat([ratings, reviews, sentiments], axis = 1)
df.columns = ['rating', 'review', 'sentiment']
df.head()

,rating,review,sentiment
0,5,I loved this place. It's conveniently located ...,1
1,4,Really wonderful evening spent at Dishoom in k...,1
2,4,"Food is legendary as ever, great value for mon...",1
3,5,I've been in London for only a week and I've a...,1
4,5,"When in London, do what the desis do, right?! ...",1


In [33]:
df

,rating,review,sentiment
0,5,I loved this place. It's conveniently located ...,1
1,4,Really wonderful evening spent at Dishoom in k...,1
2,4,"Food is legendary as ever, great value for mon...",1
3,5,I've been in London for only a week and I've a...,1
4,5,"When in London, do what the desis do, right?! ...",1
5,5,Walked in on a Saturday afternoon and we were ...,1
6,5,I've been here numerous times but oddly have n...,1
7,5,I was told this was one of THE best Indian foo...,1
8,5,Magnificence achieved! From walking in to walk...,1
9,5,Dishoom was a fantastic choice for us on our o...,1


In [34]:
stop = stopwords.words('english')
snowball = stem.snowball.EnglishStemmer()
wnl = WordNetLemmatizer()
reviews = df['review']

In [35]:
def negation_Processing(reviews):
    def neg_tag(text):
        transformed = re.sub(r"\b(?:never|nothing|nowhere|noone|none|not|haven't|hasn't|hasnt|hadn't|hadnt|can't|cant|couldn't|couldnt|shouldn't|shouldnt|won't|wont|wouldn't|wouldnt|don't|dont|doesn't|doesnt|didn't|didnt|isnt|isn't|aren't|arent|aint|ain't|hardly|seldom)\b[\w\s]+[^\w\s]", lambda match: re.sub(r'(\s+)(\w+)', r'\1NEG_\2', match.group(0)), text, flags=re.IGNORECASE)
        return(transformed)
    
    negation_reviews = []

    # Append elements to the list
    for doc in reviews:
        trans = neg_tag(doc)
        negation_reviews.append(trans)
    return negation_reviews

In [36]:
# Remove all the punctuations and numbers
def removePunc(reviews):
    comwoPunc = str.maketrans({key: None for key in string.punctuation
                               + string.digits})
    for i in reviews.keys():
        comment = reviews.loc[i]
        reviewswoPunc = reviews.replace(comment, comment.translate(comwoPunc))
    return reviewswoPunc

In [37]:
# Convert all characters to Lower case
def convToLow(words):
    reviewsToLow = words.apply(str.lower)
    return reviewsToLow

In [38]:
# Stopwords removal
def removeStop(text, stop):
    for i in text.keys():
        comment = text.loc[i]
        comment_nostop = " ".join(filter(lambda word: word not in stop,
                                     comment.split()))
        reviewswoStop = text.replace(comment, comment_nostop)
    return reviewswoStop

In [39]:
# Tokenization
def token(text):
    reviewsToken = text.apply(word_tokenize)
    return reviewsToken

In [40]:
# pos tagging
def posTag(words):
    reviews_pos = words.apply(pos_tag)
    reviews_wnpos = []
    for i in reviews_pos.keys():
        comment = reviews_pos.loc[i]
        comment_wnpos = []
        for t in comment:
            t = list(t)
            tag = t[1]
            if t[1].startswith('J'):
                t[1] = wn.ADJ
            elif t[1].startswith('V'):
                t[1] = wn.VERB
            elif t[1].startswith('N'):
                t[1] = wn.NOUN
            elif t[1].startswith('R'):
                t[1] = wn.ADV
            else:
                del t
                t = None
            if t is not None:
                comment_wnpos.append(t)
            else:
                pass
        reviews_wnpos.append(comment_wnpos)
    reviews_wnpos = pd.Series(reviews_wnpos, index=reviews_pos.keys())
    return reviews_wnpos

In [41]:
# Lemmatization
def lemma(text, wnl):
    reviews_lem = []
    for i in text.keys():
        comment = text.loc[i]
        comment_lem = []
        for t in comment:
            word = t[0]
            tag = t[1]
            t = wnl.lemmatize(word, pos=tag)
            comment_lem.append(t)
        reviews_lem.append(comment_lem)
    allReviews = []
    for j in reviews_lem:
        reviews = ' '.join(j)
        allReviews.append(reviews)
    reviewsLemma = pd.Series(allReviews, index=text.keys())
    return reviewsLemma

In [ ]:
if __name__  == '__main__':
    
    reviews = negation_Processing(reviews)
    reviews = pd.Series(reviews)

    reviews = removePunc(reviews)
    reviews = convToLow(reviews)
    reviews = removeStop(reviews, stop)
    reviews = token(reviews)
    reviews = posTag(reviews)
    final_reviews = lemma(reviews,wnl)
    ratings = df['rating']
    sentiments = df['sentiment']
    df = pd.concat([ratings, final_reviews, sentiments], axis = 1)
    df.columns = ['rating', 'review', 'sentiment']
    df.to_csv(outputfilepath, encoding='utf-8')